In [0]:
%run "/Workspace/Users/sundarasandeepteja@gmail.com/E-Commerce Analytics Medallion Architecture with GenAI/config/project_config"

In [0]:
# Databricks notebook source
# ======================================
# GOLD LAYER: AGGREGATION TABLES
# ======================================

# MAGIC %run ../config/project_config

from pyspark.sql import functions as F
from pyspark.sql.window import Window

print("🥇 GOLD LAYER: Creating Aggregation Tables")
print("=" * 60)

# Load fact and dimension tables
fact = spark.table(GOLD_FACT_TRANSACTIONS_TABLE)
dim_products = spark.table(GOLD_DIM_PRODUCTS_TABLE)
dim_customers = spark.table(GOLD_DIM_CUSTOMERS_TABLE)
dim_date = spark.table(GOLD_DIM_DATE_TABLE)

# ======================================
# 1. DAILY SALES AGGREGATION
# ======================================
print("\n📊 Creating agg_daily_sales...")

agg_daily_sales = fact \
    .filter(F.col("status") == "Completed") \
    .groupBy("transaction_date", "date_key") \
    .agg(
        F.count("transaction_id").alias("total_orders"),
        F.sum("quantity").alias("total_items"),
        F.round(F.sum("final_amount"), 2).alias("total_revenue"),
        F.round(F.sum("gross_profit"), 2).alias("total_profit"),
        F.round(F.sum("discount_amount"), 2).alias("total_discounts"),
        F.round(F.sum("shipping_cost"), 2).alias("total_shipping"),
        F.countDistinct("customer_key").alias("unique_customers"),
        F.countDistinct("product_key").alias("unique_products"),
        F.round(F.avg("final_amount"), 2).alias("avg_order_value")
    ) \
    .join(
        dim_date.select("date_key", "year", "month", "month_name", 
                        "day_of_week", "day_name", "is_weekend"),
        on="date_key",
        how="left"
    )

# Add running totals and comparisons
window_ytd = Window.partitionBy("year").orderBy("transaction_date").rowsBetween(Window.unboundedPreceding, 0)

agg_daily_sales = agg_daily_sales \
    .withColumn("revenue_ytd", F.sum("total_revenue").over(window_ytd)) \
    .withColumn("orders_ytd", F.sum("total_orders").over(window_ytd)) \
    .withColumn("_loaded_at", F.current_timestamp())

agg_daily_sales.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{GOLD_DB}.agg_daily_sales")

print(f"  ✅ Created: {GOLD_DB}.agg_daily_sales ({agg_daily_sales.count():,} rows)")

# ======================================
# 2. MONTHLY SALES AGGREGATION
# ======================================
print("\n📊 Creating agg_monthly_sales...")

agg_monthly_sales = fact \
    .filter(F.col("status") == "Completed") \
    .groupBy("transaction_year", "transaction_month") \
    .agg(
        F.count("transaction_id").alias("total_orders"),
        F.sum("quantity").alias("total_items"),
        F.round(F.sum("final_amount"), 2).alias("total_revenue"),
        F.round(F.sum("gross_profit"), 2).alias("total_profit"),
        F.countDistinct("customer_key").alias("unique_customers"),
        F.countDistinct("product_key").alias("unique_products"),
        F.round(F.avg("final_amount"), 2).alias("avg_order_value"),
        F.min("transaction_date").alias("month_start"),
        F.max("transaction_date").alias("month_end")
    )

# Add month-over-month growth
window_mom = Window.orderBy("transaction_year", "transaction_month")

agg_monthly_sales = agg_monthly_sales \
    .withColumn("prev_month_revenue", F.lag("total_revenue").over(window_mom)) \
    .withColumn("revenue_growth_pct",
        F.when(F.col("prev_month_revenue") > 0,
            F.round((F.col("total_revenue") - F.col("prev_month_revenue")) / 
                    F.col("prev_month_revenue") * 100, 2)
        ).otherwise(0)
    ) \
    .withColumn("prev_month_orders", F.lag("total_orders").over(window_mom)) \
    .withColumn("orders_growth_pct",
        F.when(F.col("prev_month_orders") > 0,
            F.round((F.col("total_orders") - F.col("prev_month_orders")) / 
                    F.col("prev_month_orders") * 100, 2)
        ).otherwise(0)
    ) \
    .withColumn("_loaded_at", F.current_timestamp())

agg_monthly_sales.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{GOLD_DB}.agg_monthly_sales")

print(f"  ✅ Created: {GOLD_DB}.agg_monthly_sales ({agg_monthly_sales.count():,} rows)")

# ======================================
# 3. CATEGORY PERFORMANCE
# ======================================
print("\n📊 Creating agg_category_performance...")

agg_category = fact \
    .filter(F.col("status") == "Completed") \
    .join(dim_products.select("product_key", "category", "subcategory", "brand"), 
          on="product_key", how="left") \
    .groupBy("category", "subcategory") \
    .agg(
        F.count("transaction_id").alias("total_orders"),
        F.sum("quantity").alias("total_items"),
        F.round(F.sum("final_amount"), 2).alias("total_revenue"),
        F.round(F.sum("gross_profit"), 2).alias("total_profit"),
        F.round(F.avg("final_amount"), 2).alias("avg_order_value"),
        F.countDistinct("customer_key").alias("unique_customers"),
        F.countDistinct("product_key").alias("unique_products")
    ) \
    .withColumn("profit_margin_pct",
        F.round(F.col("total_profit") / F.col("total_revenue") * 100, 2)
    ) \
    .withColumn("revenue_rank",
        F.dense_rank().over(Window.orderBy(F.desc("total_revenue")))
    ) \
    .withColumn("_loaded_at", F.current_timestamp())

agg_category.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{GOLD_DB}.agg_category_performance")

print(f"  ✅ Created: {GOLD_DB}.agg_category_performance ({agg_category.count():,} rows)")

# ======================================
# 4. CUSTOMER SEGMENT ANALYSIS
# ======================================
print("\n📊 Creating agg_customer_segments...")

agg_segments = fact \
    .filter(F.col("status") == "Completed") \
    .join(dim_customers.select("customer_key", "clv_segment", "region", 
                               "age_group", "churn_risk"), 
          on="customer_key", how="left") \
    .groupBy("clv_segment", "region") \
    .agg(
        F.countDistinct("customer_key").alias("customer_count"),
        F.count("transaction_id").alias("total_orders"),
        F.round(F.sum("final_amount"), 2).alias("total_revenue"),
        F.round(F.sum("gross_profit"), 2).alias("total_profit"),
        F.round(F.avg("final_amount"), 2).alias("avg_order_value"),
        F.round(F.sum("final_amount") / F.countDistinct("customer_key"), 2).alias("revenue_per_customer")
    ) \
    .withColumn("_loaded_at", F.current_timestamp())

agg_segments.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{GOLD_DB}.agg_customer_segments")

print(f"  ✅ Created: {GOLD_DB}.agg_customer_segments ({agg_segments.count():,} rows)")

# ======================================
# 5. PRODUCT PERFORMANCE
# ======================================
print("\n📊 Creating agg_product_performance...")

# Use aliasing to disambiguate product_id after join
dim_products_sel = dim_products.select(
    "product_key",
    F.col("product_id").alias("dim_product_id"),
    "product_name",
    "category",
    "brand",
    "price_tier"
)

agg_products = fact \
    .filter(F.col("status") == "Completed") \
    .join(dim_products_sel, on="product_key", how="left") \
    .groupBy(
        "product_key",
        "dim_product_id",
        "product_name",
        "category",
        "brand",
        "price_tier"
    ) \
    .agg(
        F.count("transaction_id").alias("times_ordered"),
        F.sum("quantity").alias("total_quantity_sold"),
        F.round(F.sum("final_amount"), 2).alias("total_revenue"),
        F.round(F.sum("gross_profit"), 2).alias("total_profit"),
        F.countDistinct("customer_key").alias("unique_buyers"),
        F.round(F.avg("final_amount"), 2).alias("avg_sale_amount")
    ) \
    .withColumn(
        "revenue_rank",
        F.dense_rank().over(Window.orderBy(F.desc("total_revenue")))
    ) \
    .withColumn(
        "quantity_rank",
        F.dense_rank().over(Window.orderBy(F.desc("total_quantity_sold")))
    ) \
    .withColumn(
        "is_top_seller",
        F.when(F.col("revenue_rank") <= 100, True).otherwise(False)
    ) \
    .withColumn("_loaded_at", F.current_timestamp())

agg_products.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{GOLD_DB}.agg_product_performance")

print(f"  ✅ Created: {GOLD_DB}.agg_product_performance ({agg_products.count():,} rows)")

# ======================================
# 6. HOURLY SALES PATTERN
# ======================================
print("\n📊 Creating agg_hourly_pattern...")

agg_hourly = fact \
    .filter(F.col("status") == "Completed") \
    .groupBy("transaction_hour", "is_weekend") \
    .agg(
        F.count("transaction_id").alias("total_orders"),
        F.round(F.sum("final_amount"), 2).alias("total_revenue"),
        F.round(F.avg("final_amount"), 2).alias("avg_order_value")
    ) \
    .withColumn("hour_label",
        F.when(F.col("transaction_hour") < 6, "Night (12am-6am)")
         .when(F.col("transaction_hour") < 12, "Morning (6am-12pm)")
         .when(F.col("transaction_hour") < 18, "Afternoon (12pm-6pm)")
         .otherwise("Evening (6pm-12am)")
    ) \
    .withColumn("_loaded_at", F.current_timestamp())

agg_hourly.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{GOLD_DB}.agg_hourly_pattern")

print(f"  ✅ Created: {GOLD_DB}.agg_hourly_pattern ({agg_hourly.count():,} rows)")

# ======================================
# SUMMARY
# ======================================
print("\n" + "=" * 60)
print("📊 AGGREGATION TABLES SUMMARY")
print("=" * 60)

agg_tables = [
    "agg_daily_sales",
    "agg_monthly_sales", 
    "agg_category_performance",
    "agg_customer_segments",
    "agg_product_performance",
    "agg_hourly_pattern"
]

for table in agg_tables:
    count = spark.table(f"{GOLD_DB}.{table}").count()
    print(f"  {table}: {count:,} rows")

print("\n🥇 ALL AGGREGATION TABLES COMPLETE!")